# 6.3 Sets of longer tuples

AMPL's notation for ordered pairs extends in a natural way to triples, quadruples, or
ordered lists of any length. All tuples in a set must have the same dimension. A set can't
contain both pairs and triples, for example, nor can the determination as to whether a set
contains pairs or triples be made according to some value in the data.

The multicommodity transportation model of [Figure 4-1](../04/4_1_a_multicommodity_transportation_model.ipynb#fig-4-1) offers some examples of how
we can use ordered triples, and by extension longer tuples. In the original version of the
model, the costs and amounts shipped are indexed over origin-destination-product triples:
```
param cost {ORIG,DEST,PROD} >= 0;
var Trans {ORIG,DEST,PROD} >= 0;
```
In the objective, cost and Trans are written with three subscripts, and the total cost is
determined by summing over all triples:
```
minimize Total_Cost:
   sum {i in ORIG, j in DEST, p in PROD}
      cost[i,j,p] * Trans[i,j,p];
```
The indexing expressions are the same as before, except that they list three sets instead of
two. An indexing expression that listed `k` sets would similarly denote a set of k-tuples.

If instead we define LINKS as we did in [Figure 6-2a](../06/6_2_subsets_and_slices_of_ordered_pairs.ipynb#fig-6-2a), the multicommodity declarations
come out like this:
```
set LINKS within {ORIG,DEST};
param cost {LINKS,PROD} >= 0;
var Trans {LINKS,PROD} >= 0;
minimize Total_Cost:
   sum {(i,j) in LINKS, p in PROD} cost[i,j,p] * Trans[i,j,p];
```
Here we see how a set of triples can be specified as combinations from a set of pairs
(`LINKS`) and a set of single members (`PROD`). Since `cost` and `Trans` are indexed over
`{LINKS,PROD}`, their first two subscripts must come from a pair in `LINKS`, and their
third subscript from a member of `PROD`. Sets of longer tuples can be built up in an analogous
way.

As a final possibility, it may be that only certain combinations of origins, destinations,
and products are workable. Then it makes sense to define a set that contains only the triples
of allowed combinations:
```
set ROUTES within {ORIG,DEST,PROD};
```
The costs and amounts shipped are indexed over this set:
```
param cost {ROUTES} >= 0;
var Trans {ROUTES} >= 0;
```
and in the objective, the total cost is a sum over all triples in this set:
```
minimize Total_Cost:
  sum {(i,j,p) in ROUTES} cost[i,j,p] * Trans[i,j,p];
```
Individual triples are written, by analogy with pairs, as a parenthesized and comma-separated
list `(i,j,p)`. Longer lists specify longer tuples.

In the three constraints of this model, the summations must be taken over three different
slices through the set `ROUTES`:
```
subject to Supply {i in ORIG, p in PROD}:
  sum {(i,j,p) in ROUTES} Trans[i,j,p] = supply[i,p];
subject to Demand {j in DEST, p in PROD}:
  sum {(i,j,p) in ROUTES} Trans[i,j,p] = demand[j,p];
subject to Multi {i in ORIG, j in DEST}:
  sum {(i,j,p) in ROUTES} Trans[i,j,p] <= limit[i,j];
```
In the `Supply` constraint, for instance, indices `i` and `p` are defined before the sum, so
`{(i,j,p) in ROUTES}` refers to all `j` such that `(i,j,p)` is a triple in `ROUTES`.
AMPL allows comparable slices through any set of tuples, in any number of dimensions
and any combination of coordinates.

When you declare a high-dimensional set such as `ROUTES`, a phrase like within
`{ORIG,DEST,PROD}` may specify a set with a huge number of members. With 10 origins,
100 destinations and 100 products, for instance, this set potentially has 100,000
members. Fortunately, AMPL does not create this set when it processes the declaration,
but merely checks that each tuple in the data for `ROUTES` has its first component in
`ORIG`, its second in `DEST`, and its third in `PROD`. The set `ROUTES` can thus be handled
efficiently so long as it does not itself contain a huge number of triples.

When using high-dimensional sets in other contexts, you may have to be more careful
that you do not inadvertently force AMPL to generate a large set of tuples. As an example
, consider how you could constrain the volume of all products shipped out of each origin
to be less than some amount. You might write either
```
subject to Supply_All {i in ORIG}:
  sum {j in DEST, p in PROD: (i,j,p) in ROUTES}
     Trans[i,j,p] <= supply_all[i];
```
or, using the more compact slice notation,
```
subject to Supply_All {i in ORIG}:
  sum {(i,j,p) in ROUTES} Trans[i,j,p] <= supply_all[i];
```
In the first case, AMPL explicitly generates the set `{j in DEST, p in PROD}` and
checks for membership of `(i,j,p)` in ROUTES`, while in the second case it is able to
use a more efficient approach to finding all `(i,j,p)` from `ROUTES` that have a given `i`.
In our small examples this may not seem critical, but for problems of realistic size the
slice version may be the only one that can be processed in a reasonable amount of time
and space.